In [1]:
import sys
sys.path.append('..')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from constants import *
import os
from eda_utils import *
from utils import eval_stat
import seaborn as sns
import pickle as pkl
import joblib

In [3]:
df = pd.read_pickle(f'{data_bihar}/Bihar_512_Sensor_Data_May_Aug_Screened_Hourly.pkl')

In [4]:
df

,timestamp,device_id,block,district,latitude,longitude,rh,temp,pm25
0,2023-05-01 00:00:00,1201230001,RAJPUR.B,BUXAR,25.392,83.903,NaN,NaN,NaN
1,2023-05-01 01:00:00,1201230001,RAJPUR.B,BUXAR,25.392,83.903,NaN,NaN,NaN
2,2023-05-01 02:00:00,1201230001,RAJPUR.B,BUXAR,25.392,83.903,NaN,NaN,NaN
3,2023-05-01 03:00:00,1201230001,RAJPUR.B,BUXAR,25.392,83.903,NaN,NaN,NaN
4,2023-05-01 04:00:00,1201230001,RAJPUR.B,BUXAR,25.392,83.903,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1511419,2023-08-31 19:00:00,1201230001,UJIARPUR,SAMASTIPUR,25.770,85.797,72.263846,33.670769,NaN
1511420,2023-08-31 20:00:00,1201230001,UJIARPUR,SAMASTIPUR,25.770,85.797,75.276667,33.375,NaN
1511421,2023-08-31 21:00:00,1201230001,UJIARPUR,SAMASTIPUR,25.770,85.797,77.23,33.026667,NaN
1511422,2023-08-31 22:00:00,1201230001,UJIARPUR,SAMASTIPUR,25.770,85.797,79.559872,32.780897,NaN


In [5]:
grp = df.groupby('timestamp')
df = df[['timestamp', 'latitude', 'longitude', 'rh', 'temp', 'pm25']]
len(grp)

2952

In [6]:
timestamps = []

In [7]:
cnt = 0

for name, group in grp:
    cnt += (1 if group['pm25'].count() >= 300 else 0)
    if group['pm25'].count() >= 300:
        timestamps.append(name)
cnt

1329

In [8]:
# timestamps

In [9]:
data = df[df['timestamp'].isin(timestamps)]

In [10]:
# file_name = f'{data_bihar}/Bihar_419_data_modified.csv'
# data.to_csv(file_name, header=True, index=False)
data.groupby('timestamp').count()

,latitude,longitude,rh,temp,pm25
timestamp,,,,,
2023-06-28 14:00:00,512,512,301,296,300
2023-06-28 17:00:00,512,512,301,295,300
2023-06-28 18:00:00,512,512,299,293,300
2023-06-30 18:00:00,512,512,306,302,305
2023-06-30 19:00:00,512,512,305,301,304
...,...,...,...,...,...
2023-08-31 19:00:00,512,512,407,382,360
2023-08-31 20:00:00,512,512,405,381,324
2023-08-31 21:00:00,512,512,401,376,306


In [12]:
# df['timestamp'] = df['timestamp'].values.astype(float)
data['timestamp'] = data['timestamp'].values.astype(float)
data.dtypes

/tmp/ipykernel_314511/2470082079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['timestamp'] = data['timestamp'].values.astype(float)


timestamp    float64
latitude     float64
longitude    float64
rh            object
temp          object
pm25          object
dtype: object

In [13]:
data = data.to_numpy()
data.shape

(680448, 6)

In [14]:
METHOD = 'iterative'
SPLIT = 'timestamp'
MODEL_TYPE = 'xgb'

In [15]:
imputed_data = impute(data, method=METHOD)
print(train_and_eval(imputed_data, method=METHOD, model_dir=model_dir, split=SPLIT, model_type=MODEL_TYPE))

[{'Train_RMSE': 9.336346550096163, 'Train_Pearson_R': 0.784900201431179, 'Test_RMSE': 10.445738540672341, 'Test_Pearson_R': 0.7431635953672976}]


In [30]:
SPLITS = ['random', 'lat_long', 'timestamp']
METHODS = ['knn', 'mean', 'iterative']

In [29]:
for SPLIT in SPLITS:
    print(train_and_eval(imputed_data, method=METHOD, model_dir=model_dir, split=SPLIT, model_type=MODEL_TYPE))

[{'Train_RMSE': 7.6586878494643775, 'Train_Pearson_R': 0.8644008850970275, 'Test_RMSE': 8.597848724586674, 'Test_Pearson_R': 0.8508542049859105}]
[{'Train_RMSE': 7.338908286263078, 'Train_Pearson_R': 0.8746329148369499, 'Test_RMSE': 9.434005627093422, 'Test_Pearson_R': 0.7920821571642448}]
[{'Train_RMSE': 7.338908286263078, 'Train_Pearson_R': 0.8746329148369499, 'Test_RMSE': 9.434005627093422, 'Test_Pearson_R': 0.7920821571642448}]


In [31]:
for METHOD in METHODS:
    imputed_data = impute(data, method=METHOD)
    print(train_and_eval(imputed_data, method=METHOD, model_dir=model_dir, split=SPLIT, model_type=MODEL_TYPE))

[{'Train_RMSE': 7.899726858161879, 'Train_Pearson_R': 0.8975550649824264, 'Test_RMSE': 11.918575004492297, 'Test_Pearson_R': 0.7588057040908839}]
[{'Train_RMSE': 7.364545828433064, 'Train_Pearson_R': 0.8064213589521233, 'Test_RMSE': 9.53456491461118, 'Test_Pearson_R': 0.7083230972334401}]
[{'Train_RMSE': 7.338908286263078, 'Train_Pearson_R': 0.8746329148369499, 'Test_RMSE': 9.434005627093422, 'Test_Pearson_R': 0.7920821571642448}]
